In [1]:
from edgar_functions import *
from name_mappings import (
    balance_sheet_mapping,
    income_statement_mapping,
    cash_flow_mapping,
)

tickers = [
    "AAPL",
    "MSFT",
    "AMZN",
    "NVDA",
    "GOOGL",
    "TSLA",
    "GOOG",
    "BRK-B",
    "META",
    "UNH",
    "XOM",
    "LLY",
    "JPM",
    "JNJ",
    "V",
    "PG",
    "MA",
    "AVGO",
    "HD",
    "CVX",
    "MRK",
    "ABBV",
    "COST",
    "PEP",
    "WSM",
    "ADBE",
]
ticker = tickers[5]

In [2]:
print_links_to_desired_statment(ticker, "income_statement")
print_links_to_desired_statment(ticker, "balance_sheet")
print_links_to_desired_statment(ticker, "cash_flow_statement")

balance = form_statement_for_all_available_years(ticker, "balance_sheet")
income = form_statement_for_all_available_years(ticker, "income_statement")
cash = form_statement_for_all_available_years(ticker, "cash_flow_statement")
save_dataframe_to_csv(balance, "statements", "temp", "balance_sheet", "annual")
save_dataframe_to_csv(income, "statements", "temp", "income_statement", "annual")
save_dataframe_to_csv(cash, "statements", "temp", "cash_flow", "annual")

https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000095017022000796/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459021004599/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459018002956/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459017003118/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459016013195/R4.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/R2.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000095017022000796/R2.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459021004599/R2.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/R2.htm
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/R2.htm

In [3]:
balance = pd.read_csv("statements/temp/balance_sheet_annual.csv", index_col=0)
income = pd.read_csv("statements/temp/income_statement_annual.csv", index_col=0)
cash = pd.read_csv("statements/temp/cash_flow_annual.csv", index_col=0)

In [4]:
balance_sheet = balance
income_statement = income
cash_flow_statement = cash

In [5]:
statement = income_statement
statement_mapping = income_statement_mapping.income_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31
us-gaap_RevenuesAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Revenue,81462.00,53823.00,31536.00,24578.00,21461.00,11759.00,7000132.00,4046025.00,3198356.00,2013496.00
us-gaap_CostOfRevenueAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Revenue,60609.00,40217.00,24906.00,20509.00,17419.00,9536.00,5400875.00,3122522.00,2316685.00,1557234.00
Gross Profit,20853.00,13606.00,6630.00,4069.00,4042.00,2223.00,1599257.00,923503.00,881671.00,456262.00
OPERATING EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Research and Development,3075.00,2593.00,1491.00,1343.00,1460.00,1378.00,834408.00,717900.00,464700.00,231976.00
SG&A,3946.00,4517.00,3145.00,2646.00,2835.00,2477.00,1432189.00,922232.00,603660.00,285569.00
tsla_RestructuringAndOtherExpenses,176.00,-27.00,0.00,149.00,135.00,NaN,NaN,NaN,NaN,NaN
Operating Expenses,7197.00,7083.00,4636.00,4138.00,4430.00,3855.00,2266597.00,1640132.00,1068360.00,517545.00


In [6]:
statement = balance_sheet
statement_mapping = balance_sheet_mapping.balance_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31
CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash,16253.0,17576.0,19384.0,6268.0,3686.0,3367914.0,3393216.0,1196908.0,1905713.0
Marketable Securities,5932.0,131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accounts Receivable,2952.0,1913.0,1886.0,1324.0,949.0,515381.0,499142.0,168965.0,226604.0
Inventory,12839.0,5757.0,4101.0,3552.0,3113.0,2263537.0,2067454.0,1277838.0,953675.0
...,...,...,...,...,...,...,...,...,...
us-gaap_RestrictedCashAndCashEquivalentsAtCarryingValue,NaN,NaN,NaN,NaN,NaN,NaN,105519.0,22628.0,NaN
us-gaap_DebtInstrumentConvertibleCarryingAmountOfTheEquityComponent,NaN,NaN,NaN,NaN,NaN,NaN,8784.0,47285.0,58196.0
us-gaap_RestrictedCashAndInvestmentsCurrent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22628.0,17947.0
Accrued Expenses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422798.0,268883.0


In [7]:
statement = cash_flow_statement
statement_mapping = cash_flow_mapping.cash_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31
OPERATING ACTIVITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Earnings,12587.0,5644.0,862.0,-775.0,-1063.0,-2241.0,-773046.0,-888663.0,-294040.0,NaN
ADJUSTMENTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tsla_DepreciationAmortizationAndImpairment,3747.0,2911.0,2322.0,2154.0,1901.0,1636.0,947099.0,NaN,NaN,NaN
Stock Based Compensation Expense,1560.0,2121.0,1734.0,898.0,749.0,467.0,334225.0,197999.0,156496.0,80737.0
...,...,...,...,...,...,...,...,...,...,...
us-gaap_GainLossOnDispositionOfAssets1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37723.0,14178.0,1796.0
Other Non Cash Operating Activities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26373.0,7471.0,1815.0
tsla_IncreaseDecreaseInRestrictedCashAndCashEquivalents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14752.0
Repayment of Long Term Debt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-452337.0
